In [1]:
import importlib.util
import sys
loc = '/afs/ciemat.es/user/m/martialc/public/muTel_v4/muTel/src/muTel/__init__.py'
# loc = r'F:\Clase\03 - Máster\TFM\Programa\muTel\src\muTel\__init__.py'
src = 'muTel'
spec = importlib.util.spec_from_file_location(src, loc)
foo = importlib.util.module_from_spec(spec)
sys.modules[src] = foo
spec.loader.exec_module(foo)
%load_ext autoreload


In [2]:
from IPython.display import display
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import pyarrow as pa
import os
from timeit import default_timer as timer

import muTel.utils.meta as meta
from muTel.utils.tools import flatten_dict
from muTel.utils.units import Time

from muTel.dqm.classes.MuData import MuData
from muTel.dqm.classes.MuEvent import MuEvent


In [3]:
mudata = MuData.from_datadir('run_588').sample(100)
mudata_gb = mudata.df.groupby('EventNr')
events = [MuEvent.from_data(mudata_gb.get_group(eventnr),do_muses=True) for eventnr in mudata_gb.groups.keys()]

In [4]:
events

In [5]:
muses = sum([event.all_muses for event in events],start=[])
muse_data = [muse.full_data for muse in muses]
df = pd.concat(muse_data).astype(meta.data_type_dict | {'MuSEId'    : 'category'})

In [6]:
df = df.sort_values('EventNr').reset_index().drop('index',axis='columns')
table = pa.Table.from_pandas(df)

In [7]:
writer = pq.ParquetWriter('/afs/ciemat.es/user/m/martialc/public/muTel_v4/muTel/dev/dqm/notebooks/test.parquet',table.schema)
writer.write_table(table)
writer.close()

In [8]:
muses = pq.read_table(
    '/afs/ciemat.es/user/m/martialc/public/muTel_v4/muTel/dev/dqm/muses/run_588_test.parquet'
).to_pandas().sort_values(['MuSEId','sl','layer'])
muses

,EventNr,GEO,hit,channel,sl,layer,cell,TDCtime,DriftTime,MuSEId
4,0,24,3,23,1,1,6,1741,906.25000,0A
3,0,24,2,21,1,2,6,1730,897.65625,0A
2,0,24,1,22,1,3,6,1698,872.65625,0A
5,0,24,4,20,1,4,6,1822,969.53125,0A
8,0,24,7,79,2,1,4,1732,899.21875,0B
...,...,...,...,...,...,...,...,...,...,...
794329,52223,26,3,26,3,3,7,1906,2270.31250,52223B
794334,52223,26,8,79,4,1,4,1895,2261.71875,52223C
794331,52223,26,5,77,4,2,4,1547,1989.84375,52223C
794333,52223,26,7,78,4,3,4,1863,2236.71875,52223C


In [9]:
event_df = pd.DataFrame(event_dict).transpose().sort_index()
event_df.columns = ['nhits','nmuses','n4','n3','n2','n1','MuEvent']
event_df.index.name = 'EventNr'
event_df

NameError: name 'event_dict' is not defined

In [ ]:
frac_4hits = event_df.n4.sum()/event_df.nmuses.sum()
frac_rec = (event_df.n4.sum()+event_df.n3.sum())/event_df.nmuses.sum()
frac_mult = event_df.nmuses.sum()/len(event_df)

print(
    f'Total de eventos: {len(event_df):d}\n'
    f'Total de MuSE: {event_df.nmuses.sum():d}\n'
    f'Total de MuSE reconstruibles: {event_df.n4.sum() + event_df.n3.sum():d}\n'
    f'Total de MuSE con 4 hits: {event_df.n4.sum():d}\n'
    f'Fracción de MuSE de 4 hits: {frac_4hits*100:.1f}%\n'
    f'Fracción de MuSE reconstruible: {frac_rec*100:.1f}%\n'
    f'Promedio de MuSE (por cada evento): {frac_mult:.2f}\n'
    f'Promedio de MuSE reconstruible (por cada evento): {frac_rec*frac_mult:.2f}\n'
    f'Promedio de MuSE de 4 hits (por cada evento): {frac_4hits*frac_mult:.2f}\n'
)